In [1]:
from wiki_dataset import WikiDataset
from vector_database import VectorDatabase
from torch.utils.data import DataLoader

In [2]:
wiki_dataset = WikiDataset(type='train', reduced=True, in_mem=True, num_extra_pages=10000)
wiki_dataloader = DataLoader(wiki_dataset, batch_size=8000, num_workers=10, shuffle=False)

In [3]:
len(wiki_dataset)

22549

In [4]:
vdb = VectorDatabase(client='docker', wiki_loader=wiki_dataloader)

Creating collection
Block 1/3 done
Block 2/3 done
Block 3/3 done
Time to create collection: 30.72399592399597


In [5]:
wiki_dataset[0]

{'id': '1966_Green_Bay_Packers_season',
 'text': "The Green Bay Packers season was their 47th in the National Football League . The defending NFL champions had a league-best regular season record of 12 -- 2 , led by eighth-year head coach Vince Lombardi and quarterback Bart Starr , in his eleventh NFL season .   The Packers beat the Dallas Cowboys in the NFL championship game , the Packers ' second consecutive NFL title , fourth under Lombardi , and tenth for the franchise . Two weeks later , the Packers recorded a 35 -- 10 victory over the Kansas City Chiefs in the inaugural AFL-NFL Championship Game , retroactively known as Super Bowl I.   Quarterback Starr was named the league 's most valuable player -LRB- MVP -RRB- in 1966 . Said Cold Hard Football Facts about Starr 's 1966 season , `` Starr , always underappreciated , was at his classic assassin-like best in 1966 , his lone MVP season . He led the league in completion percentage , yards per attempt and passer rating , while his 4.

In [8]:
from sentence_transformers import SentenceTransformer

wiki_dataloader = DataLoader(wiki_dataset, batch_size=100, num_workers=10, shuffle=False)

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

missing = []
for p in wiki_dataloader:
    e = model.encode(p['text'])
    r = vdb.search(e, 1)
    for i in range(len(r)):
        if r[i][0].score < 0.998:
            missing.append(p['id'][i])

In [9]:
len(missing)

0